In [1]:
import subprocess
from subprocess import PIPE, STDOUT
import json
import os
from constants import *
from dotenv import load_dotenv
from web3 import Web3, IPCProvider
from eth_account import Account
from bit import wif_to_key, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3.auto.gethdev import w3
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy

load_dotenv()
mnemonic = os.getenv("mnemonic")
type(mnemonic)

str

In [2]:
def derive_wallets(mnemonic, coin, numderive):
    
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=path,address,privkey,pubkey --coin="{coin}" --numderive="{numderive}" --format=json'

    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()

    keys = json.loads(output)
    return keys


In [3]:
derive_wallets(mnemonic, BTCTEST, 3)

[{'path': "m/44'/1'/0'/0/0",
  'address': 'mg5zLdfJgf3VKvyJ3iJ46SzTM3jDKiJx57',
  'privkey': 'cVRrpmpwf4YwNakbQt8ZpeWk7WnsG92t5yXsJrpFfSUWQiHehsAt',
  'pubkey': '038dcb0fd221442459fb3526625ab79d4380c850e5a02a4d992e89bb029ec1c197'},
 {'path': "m/44'/1'/0'/0/1",
  'address': 'n3uvXT3AU5wbEfNCJk15RZjgTzcdXmeFcZ',
  'privkey': 'cTfjXPWxb6HWCp3mvpzw2goYY3YGMCVDLwrubH9YAChZrXdogJxC',
  'pubkey': '03cb2d17b45f3d9b3ee283de9d2126b3330223e1264166f52da1645c6a4c0d88e2'},
 {'path': "m/44'/1'/0'/0/2",
  'address': 'mrteRSmc97xALeKAXqvbcb6WsdGxRXFoca',
  'privkey': 'cVPhKHbqvfxZWaKqQHc8n5uDeuikQ4BDPQrb7YKNPKadbnGy6fTy',
  'pubkey': '028beebbb80f42f487274babd59483cdbdb8bb030911f5f08018200605c7149586'}]

In [4]:
def coin():
    coin_dict = {
        'btc-test' : derive_wallets(mnemonic, BTCTEST, 3),
        'eth' : derive_wallets(mnemonic, ETH, 3)
    }

In [5]:
coin()

In [6]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    else:
        return "Sorry, could not find coin!"

In [7]:
priv_key_to_account(ETH, derive_wallets(mnemonic, ETH, 3)[0]['privkey'])

In [8]:
#from getpass import getpass

w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

private_key = os.getenv("PRIVATE_KEY")

account_one = Account.from_key(private_key)


In [9]:
def create_raw_tx(account, recipient, amount):
    gasEstimate = w3.eth.estimateGas(
        {"from": account.address, "to": recipient, "value": amount}
    )
    return {
        "from": account.address,
        "to": recipient,
        "value": amount,
        "gasPrice": w3.eth.gasPrice,
        "gas": gasEstimate,
        "nonce": w3.eth.getTransactionCount(account.address),
    }

In [10]:
def send_tx(account, recipient, amount):
    tx = create_raw_tx(account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    print(result.hex())
    return result.hex()

In [11]:
key = wif_to_key("")

# replace with different addresses
addresses = [
    "mg5zLdfJgf3VKvyJ3iJ46SzTM3jDKiJx57",
    "0x620C7C44b977C712aFEc8f476cf97349700c8DDF",
]

outputs = []

for address in addresses:
    outputs.append((address, 0.001, "btc"))

print(key.send(outputs))

ValueError: Decoded checksum b'\x00' derived from "" is not equal to hash checksum b']\xf6\xe0\xe2'.